In [410]:
# read both reviews and business datasets as rdds <DONE 
# select the business category and stars 
# join the two datasets 
# calculate the average stars

In [411]:
import pyspark 
import argparse 
import json 

# 4.1.1 E: Top n frequent words in the review text

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
# sc = pyspark.SparkContext(conf=sc_conf)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel("OFF")



In [412]:
# read in the lines and create the rdd 
review_rdd = sc.textFile("./data/review.json").map(lambda line: json.loads(line))
business_rdd = sc.textFile("./data/business.json").map(lambda line: json.loads(line))
# loading the json into a python object 


In [413]:
# print(review_rdd.take(1))
# print(business_rdd.take(1))

In [414]:
review_rdd = review_rdd.map(lambda x: (x['business_id'], x['stars']))
business_rdd = business_rdd.filter(lambda x: x['categories']).map(lambda x: (x['business_id'], x['categories'].replace(' ', '')))

In [415]:
business_rdd.take(1)

[('1SWheh84yJXfytovILXOAQ', 'Golf,ActiveLife')]

In [416]:
business_rdd = business_rdd.map(lambda x: (x[0], x[1].split(',')))

In [417]:
business_rdd.take(1)

[('1SWheh84yJXfytovILXOAQ', ['Golf', 'ActiveLife'])]

In [418]:
temp = review_rdd.join(business_rdd)

In [419]:
temp.take(1)

[('8WBHKj2davW6hhZWlUQ7DA',
  (5.0,
   ['PerformingArts',
    'Nightlife',
    'MusicVenues',
    'Stadiums&Arenas',
    'Arts&Entertainment',
    'Cinema']))]

In [420]:
business_stars = temp.map(lambda x: (x[1][0], x[1][1])) # gives us (<categories>, stars)

In [421]:
business_stars.take(1)

[(5.0,
  ['PerformingArts',
   'Nightlife',
   'MusicVenues',
   'Stadiums&Arenas',
   'Arts&Entertainment',
   'Cinema'])]

In [422]:
business_stars = business_stars.flatMap(lambda x: [(x[0], y) for y in x[1]])

In [429]:
business_stars = business_stars.map(lambda x: (x[1], x[0]))

In [430]:
business_stars.take(10)

[('PerformingArts', 5.0),
 ('Nightlife', 5.0),
 ('MusicVenues', 5.0),
 ('Stadiums&Arenas', 5.0),
 ('Arts&Entertainment', 5.0),
 ('Cinema', 5.0),
 ('PerformingArts', 5.0),
 ('Nightlife', 5.0),
 ('MusicVenues', 5.0),
 ('Stadiums&Arenas', 5.0)]

In [434]:
avg_stars = business_stars.reduceByKey(lambda x, y: x+y)

In [435]:
avg_stars.take(1)

[('Coffee&Tea', 226632.0)]

In [436]:
counts = business_stars.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x+y)

In [437]:
counts.take(1)

[('Coffee&Tea', 58969)]